In [3]:
import pandas as pd
import numpy as np
import pprint

In [6]:
file_path = "data/normalised_college_dataset.csv"
data = pd.read_csv(file_path)
data.loc[0]

Unnamed: 0                                                        0
Overall Rank                                                      1
School                           California Institute of Technology
State                                                            CA
Undergrad. Enrollment                                     0.0448952
Admission Rate                                             0.109375
Student/faculty Ratio                                             0
4-year Grad. Rate                                          0.797753
6-year Grad. Rate                                           0.62963
Quality Rank                                                     10
Total Costs                                                0.764995
Cost After Need-based Aid                                  0.233873
Need Met                                                          1
Aid From Grants                                             0.90411
Cost After Non-Need-Based Aid                   

In [7]:
final_df = data[['ACT','6-year Grad. Rate','Average Debt','Non-Need-Based Aid+']].copy()
final_df.head()

,ACT,6-year Grad. Rate,Average Debt,Non-Need-Based Aid+
0,1.000000,0.629630,0.363044,0.141414
1,0.822222,0.777778,0.450260,0.333333
2,0.955556,0.888889,0.452174,0.010101
3,1.000000,0.555556,0.202006,1.000000
4,0.755556,0.777778,0.554063,0.303030


In [11]:
def classify_data(x):
    if (x>=0.5):
        return 1
    else:
        return 0

In [12]:
final_df['ACT'] = final_df['ACT'].apply(classify_data)

In [13]:

final_df['Non-Need-Based Aid+'] = final_df['Non-Need-Based Aid+'].apply(classify_data)

In [14]:

final_df['6-year Grad. Rate'] = final_df['6-year Grad. Rate'].apply(classify_data)

In [15]:

final_df['Average Debt'] = final_df['Average Debt'].apply(classify_data)

In [16]:
final_df.head()

,ACT,6-year Grad. Rate,Average Debt,Non-Need-Based Aid+
0,1,1,0,0
1,1,1,0,0
2,1,1,0,0
3,1,1,0,1
4,1,1,1,0


In [17]:
dataset = final_df # list of transactions; each transaction is a list of items
D = list(map(set, dataset)) # set of transactions; each transaction is a list of items
print(len(dataset))

59


In [18]:
dataset_ff = dataset.values.tolist()

In [19]:
def process(df_l):
    l_f = []
    for item in df_l:
        l_t = []
        #print(item)
        for i in range(0,4):
            if(i==0):
                if(item[i]==1):
                    l_t.append('A')
            elif(i==1):
                if(item[i]==1):
                    l_t.append('B')
            elif(i==2):
                if(item[i]==1):
                    l_t.append('C')        
            else:
                if(item[i]==1):
                    l_t.append('D')
        l_f.append(l_t)
    return(l_f)   

In [20]:
dataset_ff = process(dataset_ff)
print(dataset_ff)

[['A', 'B'], ['A', 'B'], ['A', 'B'], ['A', 'B', 'D'], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B'], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B'], ['A', 'D'], ['A', 'B'], ['A', 'B', 'C'], ['A', 'B', 'C'], [], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B', 'D'], ['A', 'B', 'C'], ['A'], ['A', 'B'], ['A', 'B'], ['B', 'C'], ['A', 'B', 'C', 'D'], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B', 'C'], ['A'], ['C'], ['C', 'D'], ['A', 'B', 'C'], ['C'], ['C'], ['A', 'B', 'C'], ['C', 'D'], ['A', 'C'], ['A', 'B', 'C'], ['A', 'D'], ['A', 'B', 'C'], ['A', 'C', 'D'], ['A'], ['B', 'C'], ['A', 'C'], ['A', 'C', 'D'], ['C'], ['A', 'B', 'C'], ['C'], ['A', 'D'], ['A', 'C'], ['B', 'C'], ['C'], ['A', 'C', 'D'], ['C'], [], ['B', 'C'], ['A', 'C'], []]


In [21]:
def create_candidates(dataset, VERBOSE=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()

    if VERBOSE:
        # Print a list of all the candidate items.
        print ("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    return list(map(frozenset, c1))

In [22]:
# Generate candidate itemsets.
C1 = create_candidates(dataset_ff, VERBOSE=True) # candidate 1-itemsets

{A, B, C, D}


**Now a function is defined to prune the candidates from the database**

In [23]:
def support_prune(dataset, candidates, min_support, VERBOSE=False):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    sscnt = {} # set for support counts
    num_items=0
    for tid in dataset:
        num_items+=1
        for candidate in candidates:
            if candidate.issubset(tid):
                sscnt.setdefault(candidate, 0)
                sscnt[candidate] += 1

    #num_items = 5 #len(dataset) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    for key in sscnt:
        # Calculate the support of itemset key.
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if VERBOSE:
        for kset in retlist:
            for item in kset:
                print ("{" + str(item) + "}")
        #print
        for key in sscnt:
            print ("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

In [24]:
# Prune candidate 1-itemsets via support-based pruning to generate frequent 1-itemsets.
F1, support_data = support_prune(dataset_ff, C1, 0.8, VERBOSE=True)

{A}:  sup = 0.7288135593220338
{B}:  sup = 0.576271186440678
{D}:  sup = 0.1864406779661017
{C}:  sup = 0.6779661016949152


In [25]:
def apriori(dataset, min_support=0.5, VERBOSE=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.


    """
    C1 = create_candidates(dataset)
    D = list(map(set, dataset))
    F1, support_data = support_prune(D, C1, min_support, VERBOSE=False) # prune candidate 1-itemsets
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        Fk, supK = support_prune(D, Ck, min_support) # prune candidate itemsets
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    if VERBOSE:
        # Print a list of all the frequent itemsets.
        for kset in F:
            for item in kset:
                print( "" \
                    + "{" \
                    + "".join(str(i) + ", " for i in iter(item)).rstrip(', ') \
                    + "}" \
                    + ":  sup = " + str(round(support_data[item], 3)))

    return F, support_data

In [26]:
def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets begin evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            a.sort()
            b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return retList


In [27]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(dataset_ff, min_support=0.02, VERBOSE=True)

{C}:  sup = 0.678
{D}:  sup = 0.186
{B}:  sup = 0.576
{A}:  sup = 0.729
{C, D}:  sup = 0.102
{A, C}:  sup = 0.458
{B, C}:  sup = 0.407
{A, D}:  sup = 0.153
{B, D}:  sup = 0.051
{B, A}:  sup = 0.508
{A, C, D}:  sup = 0.068
{B, A, C}:  sup = 0.339
{B, A, D}:  sup = 0.051


Now Functions to generate Rules

In [28]:
def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.7):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.
    """
    m = len(H[0])
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.7, VERBOSE=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A frequent itemset.

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:

            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if VERBOSE:
                print( "" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence, VERBOSE=False):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([item]) for item in freq_set]

            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, VERBOSE=VERBOSE)

    return rules

In [29]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.01, VERBOSE=True)
#A:'ACT',B: '6-year Grad. Rate', C:'Average Debt', D:'Non-Need-Based Aid+'
print(H)

{D} ---> {C}:  conf = 0.545, sup = 0.102
{C} ---> {D}:  conf = 0.15, sup = 0.102
{C} ---> {A}:  conf = 0.675, sup = 0.458
{A} ---> {C}:  conf = 0.628, sup = 0.458
{C} ---> {B}:  conf = 0.6, sup = 0.407
{B} ---> {C}:  conf = 0.706, sup = 0.407
{D} ---> {A}:  conf = 0.818, sup = 0.153
{A} ---> {D}:  conf = 0.209, sup = 0.153
{D} ---> {B}:  conf = 0.273, sup = 0.051
{B} ---> {D}:  conf = 0.088, sup = 0.051
{A} ---> {B}:  conf = 0.698, sup = 0.508
{B} ---> {A}:  conf = 0.882, sup = 0.508
[(frozenset({'D'}), frozenset({'C'}), 0.5454545454545455), (frozenset({'C'}), frozenset({'D'}), 0.15000000000000002), (frozenset({'C'}), frozenset({'A'}), 0.675), (frozenset({'A'}), frozenset({'C'}), 0.627906976744186), (frozenset({'C'}), frozenset({'B'}), 0.6000000000000001), (frozenset({'B'}), frozenset({'C'}), 0.7058823529411765), (frozenset({'D'}), frozenset({'A'}), 0.8181818181818182), (frozenset({'A'}), frozenset({'D'}), 0.2093023255813954), (frozenset({'D'}), frozenset({'B'}), 0.27272727272727276), 